<a href="https://colab.research.google.com/github/daniel-mueller92/llm_mail_classifier/blob/main/llm_mail_classifier_llama2-7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install huggingface_hub

  Using cached llama_cpp_python-0.2.24.tar.gz (8.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.24-cp310-cp310-manylinux_2_35_x86_64.whl size=8156058 sha256=455a41ff3a3da3b0377c6f650b1bc0841b21ebd5626558114c28fcde5895f45c
  Stored in directory: /root/.cache/pip/wheels/9b/5f/31/172f820a61d82eb7b057b7d05ff9a05345012066e8ae781788
Successfully built llama-cpp-python


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np

import re
import json
import os

# dont do this at home
import warnings
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_excel('/content/drive/MyDrive/dataset_anonymized.xlsx')

In [10]:
def generate_prompt(df, idx, step=1):

    file_content = df["txt"][idx]

    task1 = """Bitte extrahiere folgende Informationen aus dem Dokument:
    1. Um welche Kategorie von Dokument handelt es sich? Die Auswahlmöglichkeiten sind "Deckungszusage", "Deckungsablehnung", "Nachfrage" oder "Kostensache". Bei einer
    Deckungszusage, erklärt die Versicherung, sie wird die zukünftigen Kosten des Falls übernehmen - bei einer Deckungszusage verneint sie das. Bei einer Nachfrage
    möchte die Versicherung mehr Informationen von der Kanzlei. Eine Kostensache liegt nur vor, wenn die Versicherung erklärt, sie hat bereits Kosten bezahlt.
    2. Wie lautet das Aktenzeichen der Kanzlei?
    3. Wie lautet der Name des Mandanten?
    4. Wie lautet die Schadensnummer des Versicherungsfalls?
    Wenn du einen Wert nicht findest, schreibe "None".

    Bitte formatiere deine Antwort als json-Datei wie folgt:
    {
        "aktenzeichen" : {extrahiertes Aktenzeichen},
        "name" : {extrahierter Name},
        "schadensnummer" : {extrahierte Schadensnummer},
        "kategorie" : {extrahierte Kategorie: Deckungszusage - Deckungsablehnung - Nachfrage - Kosten}
    }"""

    task_dz = """Bitte extrahiere folgende Informationen aus der Deckungszusage.
    Auf welche Instanz bezieht sich die Deckungszusage? Zur Auswahl steht das außergerichtliche Verfahren, das gerichtliche Verfahren der ersten Instanz und das
    Berufungsverfahren (2. Instanz).
    Was ist der Betrag der Selbstbeteiligung der Deckungszusage? Wenn keine Selbstbeteiligung erwähnt ist, antworte mit 0.

    Bitte formatiere deine Antwort als json-Datei wie folgt:
    {
        "außergerichtlich" : true/false,
        "gerichtlich" : true/false,
        "berufung" : true/false,
        "selbstbeteiligung" : {extrahierter Betrag}
    }
    """

    task_nachfrage = """1. Welche Nachfragegründe kannst du in der Nachfrage finden? Fragt die Rechtsschutzversicherung nach dem Sachstand? Fragt Sie nach der Klage?
    Fragt Sie nach dem Anspruchsschreiben? Erinnert Sie an ein vorheriges Schreiben? Fragt Sie nach sonstigen Fragen?

    2. Bitte füge alle gefundenen Antworten zu einer Liste zusammen.

    3. Bitte formatiere deine Antwort im json-Format wie folgt:
    {
        "Nachfragen" : [Liste der gefundenen Nachfragegründe: "Sachstand", "Klage", "Anspruchschreiben", "Erinnerung", "Sonstiges"]
    }
    """

    if step == 1:
        prompt = file_content + "-Ende der Nachricht-\n" + task1
    elif step == 2 and df["Typ"][idx] == "Deckungszusage":
        prompt = file_content + "-Ende der Deckungszusage-\n" + task_dz
    elif step == 2 and df["Typ"][idx] == "Nachfrage":
        prompt = file_content + "-Ende der Nachfrage-\n" + task_nachfrage

    return prompt

def send_request(prompt):
    response = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
          {
              "role": "user",
              "content": prompt
          }
      ],
      max_tokens=2048,
      temperature=0.2,
    )
    return response

#convert response and save dict to df
def save_response(response, idx, step):
    column_name = f'response_{step}_{model}'
    if column_name not in df.columns:
        df[column_name] = None

    # llama and mistral
    if model == "llama-13b" or model == "mistral-7b":
        try:
            response_text = response['choices'][0]['message']['content'].strip()
            reponse_json  = extract_json(response_text)

            response_dict = json.loads(reponse_json)
            df[column_name][idx] = response_dict

            if len(response_dict) > 4:
                print("Unexpected number of arguments in dict!")
            else:
                for key, value in response_dict.items():
                    if f'{key}_{model}' not in df.columns:
                        df[f'{key}_{model}'] = ''
                    df.at[idx, f'{key}_{model}'] = value

        except:
            df[column_name][idx] = response_text
            print("Antwort konnte nicht zu json formatiert werden")



def extract_json(string):
    # Define a regular expression pattern to match JSON
    pattern = r'\{(.|\n)*\}'

    # Find the JSON part using regex
    json_match = re.search(pattern, string)

    if json_match:
        extracted_json = json_match.group()
        #print(extracted_json)
    else:
        extracted_json = "No JSON found"
        #print("No JSON part found in the string.")

    return extracted_json

In [11]:
# Imports
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

In [13]:
#modelpath=hf_hub_download(repo_id='TheBloke/em_german_mistral_v01-GGUF', filename="em_german_mistral_v01.Q8_0.gguf")
modelpath=hf_hub_download(repo_id='TheBloke/Llama-2-7B-Chat-GGUF', filename="llama-2-7b-chat.Q5_K_M.gguf")

#modelpath2=hf_hub_download(repo_id='TheBloke/Llama-2-13B-chat-GGUF', filename="llama-2-13b-chat.Q5_K_M.gguf")

llama-2-7b-chat.Q5_K_M.gguf:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

In [14]:
llm = Llama(
      model_path=modelpath,
      n_ctx=2048,
      n_batch=1024,
      n_gpu_layers=-1,
      #verbose=False
      chat_format="llama-2"
      )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# workflow for llama-7b

model = "llama-7b"

for idx, row in df.iterrows():

    # first prompt/step
    prompt = generate_prompt(df, idx, step=1)
    response = send_request(prompt)

    print(response['choices'][0]['message']['content'])

    # write code to extract json only
    # and add it here
    save_response(response, idx, step=1)

    # second prompt/step only for "Deckungszusage" and "Nachfrage"
    if df["Typ"][idx] == "Deckungszusage" or df["Typ"][idx] == "Nachfrage":
        prompt2 = generate_prompt(df, idx, step=2)
        response2 = send_request(prompt2)
        # check if json only was returned
        save_response(response2, idx, step=2)

    print(f"Successfully processed entry {idx +1} of {df.shape[0]}")
    df.to_excel('/content/drive/MyDrive/results_llama-7b.xlsx')

print("All done :)")

Llama.generate: prefix-match hit


  {
    "aktenzeichen" : "S-21-03384164",
    "name" : "Denver Pearson",
    "schadensnummer" : "S-21-03384164",
    "kategorie" : "Deckungszusage"
}


Llama.generate: prefix-match hit
